# 04 angr symbolic stack

Let's analyze this assembly code:
```
                             **************************************************************
                             *                          FUNCTION                          *
                             **************************************************************
                             void __cdecl handle_user(void)
             void              <VOID>         <RETURN>
             uint32_t          Stack[0x8]:4   user_int1
             uint32_t          Stack[-0x4]:4  user_int0
             undefined8        Stack[-0x10]:8 local_10                                XREF[2]:     00100c54(W), 
                                                                                                   00100ccc(*)  
             undefined4        Stack[-0x14]:4 local_14                                XREF[3]:     00100c70(R), 
                                                                                                   00100c78(W), 
                                                                                                   00100c88(R)  
             undefined4        Stack[-0x18]:4 local_18                                XREF[3]:     00100c7c(R), 
                                                                                                   00100c84(W), 
                                                                                                   00100c9c(R)  
                             handle_user                                     XREF[4]:     Entry Point(*), 001005b0, 
                                                                                          00100660(*), main:00100cf8(c)  
        00100c50 ff 83 00 d1     sub        sp,sp,#0x20
        00100c54 fd 7b 01 a9     stp        x29,x30,[sp, #local_10]
        00100c58 fd 43 00 91     add        x29,sp,#0x10
        00100c5c 00 00 00 90     adrp       x0,0x100000
        00100c60 00 d0 15 91     add        x0=>s_%u_%u_00100574,x0,#0x574                   = "%u %u"
        00100c64 a1 13 00 d1     sub        x1,x29,#0x4
        00100c68 e2 23 00 91     add        x2,sp,#0x8
        00100c6c 39 00 00 94     bl         <EXTERNAL>::scanf                                int scanf(char * __format, ...)
        00100c70 a0 c3 5f b8     ldur       w0,[x29, #local_14]
        00100c74 ad fe ff 97     bl         complex_function0                                uint32_t complex_function0(uint3
        00100c78 a0 c3 1f b8     stur       w0,[x29, #local_14]
        00100c7c e0 0b 40 b9     ldr        w0,[sp, #local_18]
        00100c80 4f ff ff 97     bl         complex_function1                                uint32_t complex_function1(uint3
        00100c84 e0 0b 00 b9     str        w0,[sp, #local_18]
        00100c88 a8 c3 5f b8     ldur       w8,[x29, #local_14]
        00100c8c a9 be 8e 52     mov        w9,#0x75f5
        00100c90 c9 4d af 72     movk       w9,#0x7a6e, LSL #16
        00100c94 08 01 09 4a     eor        w8,w8,w9
        00100c98 c8 00 00 35     cbnz       w8,LAB_00100cb0
        00100c9c e8 0b 40 b9     ldr        w8,[sp, #local_18]
        00100ca0 c9 00 82 52     mov        w9,#0x1006
        00100ca4 c9 be a3 72     movk       w9,#0x1df6, LSL #16
        00100ca8 08 01 09 4a     eor        w8,w8,w9
        00100cac a8 00 00 34     cbz        w8,LAB_00100cc0
                             LAB_00100cb0                                    XREF[1]:     00100c98(j)  
        00100cb0 00 00 00 90     adrp       x0,0x100000
        00100cb4 00 a0 15 91     add        x0=>s_Try_again._00100568,x0,#0x568              = "Try again.\n"
        00100cb8 2a 00 00 94     bl         <EXTERNAL>::printf                               int printf(char * __format, ...)
        00100cbc 04 00 00 14     b          LAB_00100ccc
                             LAB_00100cc0                                    XREF[1]:     00100cac(j)  
        00100cc0 00 00 00 90     adrp       x0,0x100000
        00100cc4 00 74 15 91     add        x0=>s_Good_Job._0010055d,x0,#0x55d               = "Good Job.\n"
        00100cc8 26 00 00 94     bl         <EXTERNAL>::printf                               int printf(char * __format, ...)
                             LAB_00100ccc                                    XREF[1]:     00100cbc(j)  
        00100ccc fd 7b 41 a9     ldp        x29=>local_10,x30,[sp, #0x10]
        00100cd0 ff 83 00 91     add        sp,sp,#0x20
        00100cd4 c0 03 5f d6     ret
```

We will use the symbolic stack to solve this problem.

First we load the compiled binary executable:

In [1]:
import angr, claripy, sys

libname = "../build/arm64-v8a/04_angr_symbolic_stack"
project = angr.Project(libname, main_opts={ "base_addr": 0x00100000 })
print(project.arch, hex(project.entry))

<Arch AARCH64 (LE)> 0x10069c


We want start our execetion after invoke `scanf` function:(e.g.: 0x00100c70)

In [2]:
after_scanf_address = int(input("after scanf address(hex): "), 16)

We prepare the **blank state** for symbolic execution:

In [3]:
blank_state = project.factory.blank_state(addr=after_scanf_address)
blank_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_MEMORY)
blank_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_REGISTERS)

Let's create some symbolic stack which return as result by `scanf` function:
```
        00100c60 00 d0 15 91     add        x0=>s_%u_%u_00100574,x0,#0x574                   = "%u %u"
        00100c64 a1 13 00 d1     sub        x1,x29,#0x4
        00100c68 e2 23 00 91     add        x2,sp,#0x8
        00100c6c 39 00 00 94     bl         <EXTERNAL>::scanf                                int scanf(char * __format, ...)
```

Since we starting after 0x00100c6c, so we need to construct the stack ourselves, **note that we did not use `stack_push`**, we use `memory.store` instead for arm64.

In [4]:
blank_state.regs.sp -= 0x20
blank_state.regs.x29 = blank_state.regs.sp + 0x10

password0 = claripy.BVS('password0', 32)
blank_state.memory.store(blank_state.regs.x29 - 0x4, password0, size=4, endness=blank_state.arch.memory_endness) # x29-0x4 = sp+0xc

password1 = claripy.BVS('password1', 32)
blank_state.memory.store(blank_state.regs.sp + 0x8, password1, size=4, endness=blank_state.arch.memory_endness)

Everything is ready, next we start the symbolic execution to find the path:

In [5]:
def is_successful(state):
    stdout_output = state.posix.dumps(1) # stdout
    return "Good Job.".encode() in stdout_output 

def should_abort(state):
    stdout_output = state.posix.dumps(1) # stdout
    return "Try again.".encode() in stdout_output

simulation = project.factory.simulation_manager(blank_state)
simulation.explore(find=is_successful, avoid=should_abort)
if simulation.found:
    solution0 = simulation.found[0].solver.eval(password0)
    solution1 = simulation.found[0].solver.eval(password1)

    solution = ' '.join(map('{:d}'.format, [ solution0, solution1 ])) 
    print("Solution(password):", solution)
else:
    print("Cound not find the solution")
    list(map(lambda s: s.callstack.dbg_repr(), simulation.deadended))

Solution(password): 833191447 3394749455
